# Aaron's (minimal) example of using Firefly with Gaia data


## Download the Gaia data 

In [102]:
import numpy as np
from scipy.signal import find_peaks

from astroquery.gaia import Gaia

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table

import matplotlib.pyplot as plt
%matplotlib inline

Retrieve all the available data in the region of interest.

I will download data in the direction of the open cluster M67 (coordinates: RA = 132.825 deg, +11.8167) with a search radius of 1 degrees.

I will perform an asynchronous query (asynchronous rather than synchronous queries should be performed when retrieving more than 2000 rows), and also require that the parallax and proper motion data to be well behaved (to remove likely spurious data).

Note: The query to the archive is with ADQL (Astronomical Data Query Language). For a description of ADQL and more examples see the Gaia DR1 ADQL cookbook: https://gaia.ac.uk/data/gaia-data-release-1/adql-cookbook

In [ ]:
cmd = "SELECT * FROM gaiadr3.gaia_source \
    WHERE CONTAINS(POINT('ICRS',gaiadr3.gaia_source.ra, gaiadr3.gaia_source.dec),\
    CIRCLE('ICRS', 132.825, 11.8167, 1))=1\
    AND parallax>0 \
    AND abs(pmra_error)<5 \
    AND abs(pmdec_error)<5 \
    AND pmra IS NOT NULL AND abs(pmra)>0 \
    AND pmdec IS NOT NULL AND abs(pmdec)>0;"

job = Gaia.launch_job_async(cmd, dump_to_file=False) #could save this to a file

print (job)

This downloads as an astropy table.  Save this to an [ecsv](https://docs.astropy.org/en/stable/io/ascii/ecsv.html) file (to retain the units and masks).  That way I can reuse this data later without having to download from Gaia

In [24]:
tab = job.get_results()
tab.write('Gaia_m67.ecsv', overwrite=True)

In [17]:
# to read the data back in
# tab = Table.read('Gaia_m67.ecsv')

Calculate 3D coordinates

In [ ]:
coords_3d = SkyCoord(
    ra=tab['ra'], 
    dec=tab['dec'],
    distance=Distance(parallax=tab['parallax'].filled(np.nan))
)
coords_3d.cartesian

Do the same for the expected cluster center (from literature/internet)

In [55]:
coords_3d_center = SkyCoord(
    ra=132.825*u.deg, 
    dec=11.8167*u.deg,
    distance=900*u.parsec
)

I prefer pandas (and so does Firefly), though note that this will remove the units.  Let's also simply things to only continue with the columns we're interested in.

In [ ]:
# look at all the column names
list(tab.columns)

In [ ]:
cols = [
    'SOURCE_ID',
    'ra',
    'dec',
    'parallax',
    'pmra',
    'pmdec',
    'radial_velocity',
    'phot_g_mean_mag',
    'bp_rp',
    'ruwe',
    'teff_gspphot'
]
df = tab[cols].to_pandas()

# add the 3D coordinates but center them on the expected cluster center
df['x'] = coords_3d.cartesian.x - coords_3d_center.cartesian.x
df['y'] = coords_3d.cartesian.y - coords_3d_center.cartesian.y
df['z'] = coords_3d.cartesian.z - coords_3d_center.cartesian.z

df

Possibly make a selection so that we have an additional group of only members.  (This is a rather crude estimate.)

In [134]:
def find_gaia_peak(arr, bins=100, range=[-20,20], return_plot = False):

    # calculate the histogram and find the most prominent peak (using scipy)
    heights, bin_edges = np.histogram(arr, bins=bins, range=range)
    bin_centers = bin_edges[:-1] + np.diff(bin_edges)
    pk,dct = find_peaks(heights, prominence=(None,None))
    pk = pk[dct['prominences'].argsort()[::-1]]

    peak = bin_centers[pk[0]]
    out = {'peak':peak}

    if (return_plot):
        f, ax = plt.subplots()
        ax.step(bin_centers, heights, where='mid', color='black')
        ax.axvline(peak, color='firebrick')
        out['f'] = f
        out['ax'] = ax
    
    return out

In [ ]:
dct = find_gaia_peak(df['pmdec'], bins=100, range=[-20,20], return_plot = True)
dct['peak']

In [ ]:
# I just used a guess-and-check method to settle on the numbers here
members = df.loc[
    (abs(df['parallax'] - find_gaia_peak(df['parallax'], bins=100, range=[0,5])['peak']) < 5) & 
    (abs(df['pmra'] - find_gaia_peak(df['pmra'], bins=100, range=[-20,20])['peak']) < 0.25) &
    (abs(df['pmdec'] - find_gaia_peak(df['pmdec'], bins=100, range=[-20,20])['peak']) < 0.25)
]
members

In [ ]:
# check the CMD
f, ax = plt.subplots()
ax.scatter(df['bp_rp'],df['phot_g_mean_mag'], s=1, color='gray')
ax.scatter(members['bp_rp'],members['phot_g_mean_mag'], s=1, color='firebrick')

plt.gca().invert_yaxis()

## Format Data for Firefly

In [158]:
coords_all = df[['x','y','z']].to_numpy()
coords_members = members[['x','y','z']].to_numpy()
coords = [coords_all, coords_members]

In [159]:
field_names = ['SOURCE_ID','parallax','pmra','pmdec','radial_velocity','phot_g_mean_mag','bp_rp','teff_gspphot','ruwe']
fields_all = np.nan_to_num(df[field_names], nan=-999)
fields_members = np.nan_to_num(members[field_names], nan=-999)
fields = [fields_all, fields_members]

Here are the [docs for ArrayReader](https://firefly.rcs.northwestern.edu/docs/reference/api/readers/firefly.data_reader.ArrayReader.html).

In [60]:
from firefly.data_reader import ArrayReader

In [ ]:
my_arrayReader = ArrayReader(
    coords,
    fields=fields,
    field_names=field_names,
    UInames=['all','members'],
    write_to_disk=False)

## Display Inline

In [50]:
from firefly.server import spawnFireflyServer,quitAllFireflyServers

In [51]:
# define the local port (typically anything in 5000 - 8000 range)
port = 5500

In [ ]:
process = spawnFireflyServer(port, method = 'flask')

In [ ]:
from IPython.display import IFrame
url = f'http://localhost:{port:d}/combined'
IFrame(url, width=1000, height=500)

In [ ]:
# Send data to the server.
# Wait until it loads to run this command
my_arrayReader.sendDataViaFlask()

## Get the selected data in Python

(after using the selection tool)

In [10]:
import requests

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# send a get request to receive the current settings from Firefly
# for larger amounts of data, you will need to increase the waitTime (in seconds) via params (see below; the default is 10s)
r = requests.get(url = f'http://localhost:{port:d}/get_selected_data', params = {'waitTime':60})
if r.status_code == 200:
    # success
    selection = r.json()
    # as a check
    partsKeys = list(selection.keys())
    print(partsKeys[0])
    print(selection[partsKeys[0]]['Coordinates_flat'][:100]) 
else:
    print('Error: {}'.format(r.status_code), r.content)


In [ ]:
# plot x, y for the selected points
partsKeys = list(selection.keys())
part0 = selection[partsKeys[0]]

# UPDATE THIS
x = part0['Coordinates_flat'][0::3]
y = part0['Coordinates_flat'][1::3]

f, ax = plt.subplots()
ax.scatter(x[:1000],y[:1000])

### Quit the Firefly server

... this doesn't always work in a notebook ... you can also quit the server by resetting the kernel.

In [ ]:
return_code = quitAllFireflyServers()